# **Cultura e Práticas em DataOps e MLOps**
**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook apresenta um exemplo de uma rede neural profunda com mais de uma camada para um problema de classificação.


# **Saúde Fetal**

As Cardiotocografias (CTGs) são opções simples e de baixo custo para avaliar a saúde fetal, permitindo que os profissionais de saúde atuem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais.

Este conjunto de dados contém 2126 registros de características extraídas de exames de Cardiotocografias, que foram então classificados por três obstetras especialistas em 3 classes:

- Normal
- Suspeito
- Patológico

# Instalando pacotes

In [1]:
%%capture
%pip install mlflow

# 1 - Importando os módulos necessários

In [ ]:
%pip install tensorflow

import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential # pyright: ignore[reportMissingImports]
from tensorflow.keras.layers import Dense, InputLayer # pyright: ignore[reportMissingImports]

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Note: you may need to restart the kernel to use updated packages.


# Definindo funções adicionais

In [3]:
def reset_seeds() -> None:
  """
  Resets the seeds to ensure reproducibility of results.

  This function sets the seed for various random number generation libraries
  to ensure that results are reproducible. The affected libraries are:
  - Python's built-in `random`
  - NumPy
  - TensorFlow

  The seed used is 42.

  Returns:
      None
  """
  os.environ['PYTHONHASHSEED']=str(42)
  tf.random.set_seed(42)
  np.random.seed(42)
  random.seed(42)

# 2 - Fazendo a leitura do dataset e atribuindo às respectivas variáveis

In [4]:
url = 'raw.githubusercontent.com'
username = 'renansantosmendes'
repository = 'lectures-cdas-2023'
file_name = 'fetal_health_reduced.csv'
data = pd.read_csv(f'https://{url}/{username}/{repository}/master/{file_name}')

# Dando uma leve olhada nos dados

In [5]:
data.head()

,severe_decelerations,accelerations,fetal_movement,uterine_contractions,fetal_health
0,0.0,0.0,0.0,0.0,2.0
1,0.0,6.0,0.0,6.0,1.0
2,0.0,3.0,0.0,8.0,1.0
3,0.0,3.0,0.0,8.0,1.0
4,0.0,7.0,0.0,8.0,1.0


# 3 - Preparando o dado antes de iniciar o treino do modelo

In [6]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"]

columns_names = list(X.columns)
scaler = preprocessing.StandardScaler()
X_df = scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

y_train = y_train -1
y_test = y_test - 1

# 4 - Criando o modelo e adicionando as camadas

In [7]:
reset_seeds()
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], )))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# 5 - Compilando o modelo


In [8]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

##**Configurando o mlflow**

In [9]:
import mlflow

os.environ['MLFLOW_TRACKING_USERNAME'] = 'renansantosmendes'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '6d730ef4a90b1caf28fbb01e5748f0874fda6077'
mlflow.set_tracking_uri('https://dagshub.com/renansantosmendes/puc_lectures_mlops.mlflow')

mlflow.keras.autolog(log_models=True,
                     log_input_examples=True,
                     log_model_signatures=True)

2025/09/11 06:48:17 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.13.1 <= tensorflow, but the installed version is 2.10.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


# 6 - Executando o treino do modelo

In [10]:
with mlflow.start_run(run_name='experiment_mlops_ead') as run:
  model.fit(X_train,
            y_train,
            epochs=50,
            validation_split=0.2,
            verbose=3)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


2025/09/11 06:48:35 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


🏃 View run experiment_mlops_ead at: https://dagshub.com/renansantosmendes/puc_lectures_mlops.mlflow/#/experiments/0/runs/08876d88cbd1497b8ffc8e12ff22c124
🧪 View experiment at: https://dagshub.com/renansantosmendes/puc_lectures_mlops.mlflow/#/experiments/0
